## Create and import order data to Postgresql DB

In [20]:
import pandas as pd
import numpy as np

### Load data from csv

In [21]:
# Read the CSV file
df = pd.read_csv('/mnt/c/Projects/pipelines/data/order.csv',
                 dtype={
                            'company_code': str,
                            'source_store_key': str,
                            'source_store_name': str,
                            'order_line_id': "Int64",
                            'order_id': "Int64",
                            'order_return_id': "Int64",
                            'order_status': str,
                            'sub_status_code': str,
                            'created_by': str,
                            'channel_code': str,
                            'sub_status': str,
                            'customer_phone_number': str,
                            'price_rule_order_line_title': str,
                            'price_rule_order_title': str,
                            'special_order_type': str,
                            'reason': str,
                            'discount_code_order_line': str,
                            'discount_code_order': str,
                            'variant_id': "Int64",
                            'quantity': "Int64",
                            'customer_id': "Int64",
                            'price_rule_order_line_id': "Int64",
                            'price_rule_order_id': "Int64",
                            'total': float,
                            'total_after_discount': float,
                            'total_discount_amount': float,
                            'product_discount': float,
                            'tax_line': float,
                            'product_amount_after_discount': float,
                            'order_amount_after_discount': float,
                            'net_amount_no_vat': float,
                            'price': float,
                            'price_in_order': float,
                            'paid_amount': float,
                            'attributed_source_store_key': str,
                            'attributed_channel_code': str
                        },
                        parse_dates=['order_created_time_07', 'order_finished_time_07'])



In [22]:
# check missing values
df.isnull().sum()

# check column with missing values > 0
df.columns[df.isnull().sum() > 0]

Index(['order_finished_time_07', 'sub_status_code', 'sub_status',
       'price_rule_order_line_title', 'price_rule_order_title',
       'special_order_type', 'reason', 'discount_code_order_line',
       'discount_code_order', 'price_rule_order_line_id',
       'price_rule_order_id', 'attributed_source_store_key',
       'attributed_channel_code'],
      dtype='object')

In [23]:
from sqlalchemy import create_engine, Column, Integer, String, Date, Boolean, text, DateTime, ForeignKey, Float, inspect
from sqlalchemy.orm import sessionmaker, declarative_base, relationship
import os

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

# Create a connection to the PostgreSQL database
engine = create_engine(f"postgresql+psycopg2://{os.environ['PG_USER']}:{os.environ['PG_PASSWORD']}@{os.environ['PG_HOST']}:{os.environ['PG_PORT']}/{os.environ['PG_DB']}")

# Create a base class for declarative class definitions
Base = declarative_base()


In [24]:
class Order(Base):
    __tablename__ = 'orders'

    id = Column(Integer, primary_key=True)
    company_code = Column(String)
    source_store_key = Column(String)
    source_store_name = Column(String)
    order_created_time_07 = Column(DateTime)
    order_finished_time_07 = Column(DateTime, nullable=True)
    order_id = Column(String, unique=True)
    order_return_id = Column(String)
    order_status = Column(String)
    sub_status_code = Column(String, nullable=True)
    created_by = Column(String)
    channel_code = Column(String)
    sub_status = Column(String, nullable=True)
    customer_phone_number = Column(String)
    price_rule_order_title = Column(String, nullable=True)
    special_order_type = Column(String, nullable=True)
    reason = Column(String)
    discount_code_order = Column(String, nullable=True)
    customer_id = Column(Integer)
    price_rule_order_id = Column(String, nullable=True)
    total = Column(Float)
    total_after_discount = Column(Float)
    total_discount_amount = Column(Float)
    order_amount_after_discount = Column(Float)
    paid_amount = Column(Float)
    attributed_source_store_key = Column(String, nullable=True)
    attributed_channel_code = Column(String, nullable=True)

    # Relationship
    order_lines = relationship("OrderLine", back_populates="order")

In [25]:
# Define the OrderLine table
class OrderLine(Base):
    __tablename__ = 'order_lines'

    id = Column(Integer, primary_key=True)
    order_line_id = Column(String, unique=True)
    order_id = Column(String, ForeignKey('orders.order_id'))
    price_rule_order_line_title = Column(String, nullable=True)
    discount_code_order_line = Column(String, nullable=True)
    variant_id = Column(String)
    quantity = Column(Integer)
    price_rule_order_line_id = Column(String, nullable=True)
    product_discount = Column(Float)
    tax_line = Column(Float)
    product_amount_after_discount = Column(Float)
    net_amount_no_vat = Column(Float)
    price = Column(Float)
    price_in_order = Column(Float)

    # Relationship
    order = relationship("Order", back_populates="order_lines")

In [26]:
# Create the table in the database
Base.metadata.create_all(engine)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()


In [27]:
# # drop table orders and order_lines
# session.execute(text('DROP TABLE IF EXISTS orders CASCADE'))
# session.execute(text('DROP TABLE IF EXISTS order_lines CASCADE'))
# session.commit()

In [28]:
df.dtypes

company_code                                  object
source_store_key                              object
source_store_name                             object
order_created_time_07            datetime64[ns, UTC]
order_finished_time_07           datetime64[ns, UTC]
order_line_id                                  Int64
order_id                                       Int64
order_return_id                                Int64
order_status                                  object
sub_status_code                               object
created_by                                    object
channel_code                                  object
sub_status                                    object
customer_phone_number                         object
price_rule_order_line_title                   object
price_rule_order_title                        object
special_order_type                            object
reason                                        object
discount_code_order_line                      

In [29]:
df2 = df.astype(object).where(pd.notnull(df), None)
df2.head()

,company_code,source_store_key,source_store_name,order_created_time_07,order_finished_time_07,order_line_id,order_id,order_return_id,order_status,sub_status_code,...,product_discount,tax_line,product_amount_after_discount,order_amount_after_discount,net_amount_no_vat,price,price_in_order,paid_amount,attributed_source_store_key,attributed_channel_code
0,YODY,ST244,"YODY 495 NGUYỄN THỊ THẬP, HCM",2024-02-28 12:20:02+00:00,2024-03-03 07:53:44+00:00,110602639,33108375,-1,finished,shipped,...,59800.0,17719.0,239200.0,956800.0,221481.0,299000.0,299000.0,239200.0,None,None
1,YODY,ST244,"YODY 495 NGUYỄN THỊ THẬP, HCM",2024-02-28 12:20:02+00:00,2024-03-03 07:53:44+00:00,110602640,33108375,-1,finished,shipped,...,59800.0,17719.0,239200.0,956800.0,221481.0,299000.0,299000.0,239200.0,None,None
2,YODY,ST244,"YODY 495 NGUYỄN THỊ THẬP, HCM",2024-02-28 12:20:02+00:00,2024-03-03 07:53:44+00:00,110602641,33108375,-1,finished,shipped,...,59800.0,17719.0,239200.0,956800.0,221481.0,299000.0,299000.0,239200.0,None,None
3,YODY,ST244,"YODY 495 NGUYỄN THỊ THẬP, HCM",2024-02-28 12:20:02+00:00,2024-03-03 07:53:44+00:00,110602638,33108375,-1,finished,shipped,...,59800.0,17719.0,239200.0,956800.0,221481.0,299000.0,299000.0,239200.0,None,None
4,YODY,ST244,"YODY 495 NGUYỄN THỊ THẬP, HCM",2024-03-03 09:06:33+00:00,2024-03-03 09:06:33+00:00,110763078,33159295,-1,finished,shipped,...,0.0,4370.0,59000.0,409550.0,54630.0,59000.0,59000.0,59000.0,None,None


In [30]:

# Group by order_id to process orders
for order_id, group in df2.groupby('order_id'):
    # Create Order instance
    order_data = group.iloc[0]
    order = Order(
        company_code=order_data['company_code'],
        source_store_key=order_data['source_store_key'],
        source_store_name=order_data['source_store_name'],
        order_created_time_07=order_data['order_created_time_07'],
        order_finished_time_07=order_data['order_finished_time_07'],
        order_id=order_id,
        order_return_id=order_data['order_return_id'],
        order_status=order_data['order_status'],
        sub_status_code=order_data['sub_status_code'],
        created_by=order_data['created_by'],
        channel_code=order_data['channel_code'],
        sub_status=order_data['sub_status'],
        customer_phone_number=order_data['customer_phone_number'],
        price_rule_order_title=order_data['price_rule_order_title'],
        special_order_type=order_data['special_order_type'],
        reason=order_data['reason'],
        discount_code_order=order_data['discount_code_order'],
        customer_id=order_data['customer_id'],
        price_rule_order_id=order_data['price_rule_order_id'],
        total=order_data['total'],
        total_after_discount=order_data['total_after_discount'],
        total_discount_amount=order_data['total_discount_amount'],
        order_amount_after_discount=order_data['order_amount_after_discount'],
        paid_amount=order_data['paid_amount'],
        attributed_source_store_key=order_data['attributed_source_store_key'],
        attributed_channel_code=order_data['attributed_channel_code']
    )
    session.add(order)

    # # Create OrderLine instances
    for _, line_data in group.iterrows():
        order_line = OrderLine(
            order_line_id=line_data['order_line_id'],
            order_id=order_id,
            price_rule_order_line_title=line_data['price_rule_order_line_title'],
            discount_code_order_line=line_data['discount_code_order_line'],
            variant_id=line_data['variant_id'],
            quantity=line_data['quantity'],
            price_rule_order_line_id=line_data['price_rule_order_line_id'],
            product_discount=line_data['product_discount'],
            tax_line=line_data['tax_line'],
            product_amount_after_discount=line_data['product_amount_after_discount'],
            net_amount_no_vat=line_data['net_amount_no_vat'],
            price=line_data['price'],
            price_in_order=line_data['price_in_order']
        )
        session.add(order_line)

# Commit the changes and close the session
session.commit()
session.close()

print("Data import completed successfully!")

Data import completed successfully!


In [31]:
# open session and count the number of rows in the orders table
session = Session()
orders_count = session.query(Order).count()
orders_count

26294

In [32]:
# Your SQL string
sql = """
SELECT COUNT(*) FROM orders
WHERE order_status = :param
"""

sql = """
SELECT COUNT(*) FROM order_lines
"""

# select top 10 * from orders where order_status = 'finished'

sql = """
SELECT * FROM orders
WHERE order_status = :param
LIMIT 10
"""

# Parameters for the SQL query (if any)
params = {"param": "finished"}

# Execute the SQL
with engine.connect() as connection:
    result = connection.execute(text(sql), params)
    
    # Fetch all results
    rows = result.fetchone()

    # Print results
    for row in rows:
        print(row)

1
YODY
ST244
YODY 495 NGUYỄN THỊ THẬP, HCM
2023-07-01 19:55:19
2023-07-01 19:55:19
28642025
2190742
finished
None
YD16175
POS
None
0932474708
None
None
None
None
199848
None
-399000.0
-279300.0
-119700.0
-279300.0
-279300.0
None
None
